In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

/Users/pdal-mol/work/42AI/miniconda3/envs/Hand2Text/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Hyperparameters

In [2]:
batch_size=64
learning_rate = 0.1

## Download Datasets

In [3]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('../data', train=False, download=True, transform=transform)

## Load Data

In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True)

## Model

In [5]:
class CNN(nn.Module):

	def __init__(self):
		super().__init__()
		self.conv1 = nn.Conv2d(1, 6, 5)
		self.pool = nn.MaxPool2d(2, 2)
		self.conv2 = nn.Conv2d(6, 16, 5)

		self.fc1 = nn.Linear(16*4*4, 120)
		self.fc2 = nn.Linear(120, 84)
		self.fc3 = nn.Linear(84, 10)

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = torch.flatten(x, 1)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		return x

In [6]:
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train(train_loader, model, loss_fn, optmizer):
	size = len(train_loader.dataset)
	batches_l = len(train_loader)
	loss = 0
	correct = 0

	for batch_idx, (data, target) in enumerate(train_loader):
		pred = model(data)
		loss = loss_fn(pred, target)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if batch_idx % 100 == 0:
			print(f'loss: {loss}')

In [8]:
def test(test_loader, model, loss_fn):
	size = len(test_loader.dataset)
	loss = 0
	correct_n = 0
	correct = 0

	for batch_idx, (data, target) in enumerate(test_loader):
		pred = model(data)
		argmax = pred.argmax(dim=1, keepdim=True)
		correct_n = argmax.eq(target.view_as(argmax)).sum().item()
		correct += correct_n
	return correct / size


In [9]:
train(train_loader, model, loss_fn, optimizer)

loss: 2.3090310096740723
loss: 1.7292048931121826
loss: 0.40135541558265686
loss: 0.2500225007534027
loss: 0.3237955570220947
loss: 0.15470768511295319
loss: 0.09648774564266205
loss: 0.17330554127693176
loss: 0.06466846913099289
loss: 0.2069837898015976


In [10]:
print(f'accuracy: {test(test_loader, model, loss_fn)}')

accuracy: 0.9686
